# Test

------------------------------------------

In [1]:
# from BuildAnimation import *

# ab = AnimationBuilder(build_single_person_scene_two_poses)

# ab.maya_controller.UndoToBeginning(max_step=1000)

# ab.BuildScene()

# ab.BuildDefaultFrame()

# endFrame_node = ab.pg.node_dict["end frame"]

# endFrame_node.next_frame

# ab.maya_controller.SetCurrentTimeFrame(endFrame_node.next_frame)

# ab.BuildFrame(endFrame_node)

# ab.maya_controller.ScreenShot("E:/Temp/Test0")

# ab.pg.node_dict

# endBody = ab.pg.node_dict['end body']

# endBody.chest_leaf.selected_file

# Test AOG2

In [2]:
from Raven.BuildTree2 import *

In [3]:
scene = build_scene()

In [4]:
scene.node_dict

{}

In [5]:
pg = scene.sample()

In [6]:
pg.node_dict

{'init frame': Frame.init frame,
 'transform': Layout.transform,
 'relation': Layout.relation,
 'animation1': Animation.animation1,
 'motion1': Layout.motion1,
 'emotion1': Layout.emotion1,
 'motion2': Layout.motion2,
 'emotion2': Layout.emotion2,
 'animation2': Animation.animation2}

In [7]:
pg.node_dict['transform'].position_leaf.relativePosition

[10, 2]

In [8]:
print(pg.node_dict['relation'].generation)
print(pg.node_dict['relation'].relation)

elder
society


In [9]:
print(pg.node_dict['motion1'].selected_motion)
print(pg.node_dict['motion1'].start_time)
print(pg.node_dict['motion2'].selected_motion)
print(pg.node_dict['motion2'].start_time)

Salute
1.5369231099594862
Acknowledging
1.9825480776170759


In [10]:
print(pg.node_dict['emotion1'].start_emotion)
print(pg.node_dict['emotion1'].end_emotion)
print(pg.node_dict['emotion1'].start_time)
print(pg.node_dict['emotion2'].start_emotion)
print(pg.node_dict['emotion2'].end_emotion)
print(pg.node_dict['emotion2'].start_time)

Sad
Neutral
-23.833974703201683
Dissatisfied
Mock
-17.163012815051697


In [11]:
import os
os.getcwd()

'E:\\researches\\EQTest\\SocialAffordance'

In [12]:
# VAD_DICT = {}

# with open("../data/NRC-VAD-Lexicon.txt") as f:
#     for c, line in enumerate(f.readlines()):
#         if c == 0:
#             continue
#         line_info = line.strip().split("\t")
#         #print(line_info)
#         VAD_DICT[line_info[0]] = [float(line_info[1]), float(line_info[2]), float(line_info[3])]

In [13]:
get_relation_emotion_scores(pg, G_EMOTION2VAD)

[0.125663, 0.22385100000000002]

In [14]:
get_relation_motion_scores(pg, G_MOTION2VAD)

[1.2810000000000001, 1.2610000000000001]

In [15]:
get_motion_emotion_scores(pg, G_MOTION2VAD, G_EMOTION2VAD)

[0.26305799999999996, 0.482745]

In [16]:
def load_survey(label_path):
    survey = []
    with open(label_path, "r") as f:
        for c, line in enumerate(f.readlines()):
            line_info = line.strip().split(",")
            #print(c)
            sample_info = line_info[0].split("_")
            #print(item)
            item = {"m1": sample_info[0], "e11": sample_info[1], "e12": sample_info[2],
                   "m2": sample_info[3], "e21": sample_info[4], "e22": sample_info[5],
                   "d": sample_info[6], "t": sample_info[7]}
            label_info = line_info[1].split("_")
            item["quality"] = label_info[0]
            item["dominance"] = label_info[1]
            item["intimacy"] = label_info[2]

            survey.append(item)
    
    return survey

In [17]:
survey = load_survey("../data/label2.txt")

In [18]:
survey[1]

{'m1': 'Wiping',
 'e11': 'Tired',
 'e12': 'Surprise',
 'm2': 'Whatever',
 'e21': 'Confused',
 'e22': 'Sad',
 'd': '4',
 't': '1',
 'quality': 'Good',
 'dominance': 'Character two',
 'intimacy': 'Close'}

In [19]:
get_relation_emotion_scores_survey(survey[1], G_EMOTION2VAD)

[0.37572500000000003, 0.12925]

In [20]:
get_relation_motion_scores_survey(survey[1], G_MOTION2VAD)

[-0.21766666666666656, 0.20833333333333348]

In [21]:
get_motion_emotion_scores_survey(survey[1], G_MOTION2VAD, G_EMOTION2VAD)

[0.8583910000000001, -0.13824]

In [22]:
get_time_scores(pg)

[-1.0182293391045865]

In [23]:
get_time_scores_survey(survey[1])

[-1.4189385332046727]

In [51]:
import torch
import numpy as np

In [56]:
thetas = torch.randn(7)

In [57]:
thetas

tensor([-1.3897, -1.0315, -0.8528, -1.4636, -0.2303,  0.0576,  1.2019])

In [58]:
#s_pg = torch.Tensor(get_pg_score_list(pg))

In [59]:
#s_survey = torch.Tensor(get_survey_score_list(survey[0]))

In [70]:
for i in range(100):
    while True:
        item = np.random.choice(survey)
        if item['quality'] == "Good":
            break
    
    pg = scene.sample()
    s_pg = torch.Tensor(get_pg_score_list(pg))
    s_survey = torch.Tensor(get_survey_score_list(survey[0]))
    thetas = thetas + 0.1 * (s_pg -s_survey)

In [71]:
thetas

tensor([-2.9413e+00, -6.7144e+00,  2.8066e-01,  1.1938e-02,  4.0875e-01,
         1.4030e+00,  1.4542e+01])

In [75]:
survey_good_scores = []
survey_bad_scores = []
for i in range(100,200):
    item = survey[i]
    item_score = torch.Tensor(get_survey_score_list(item)).dot(thetas).item()
    if item['quality'] == "Good":
        survey_good_scores.append(item_score)
    elif item['quality'] == "Bad":
        survey_bad_scores.append(item_score)

In [76]:
np.mean(survey_bad_scores)

-24.222471237182617

In [77]:
np.mean(survey_good_scores)

-27.957227616083053